In [4]:
import openfoamcloud.providers.google.google_provider as google_provider
cloud_provider = google_provider.GoogleCloud()



### Example to donwload full blob

In [2]:
from google.cloud import storage


def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your GCS object
    # source_blob_name = "storage-object-name"

    # The path to which the file should be downloaded
    # destination_file_name = "local/path/to/file"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    # Construct a client side representation of a blob.
    # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
    # any content from Google Cloud Storage. As we don't need additional data,
    # using `Bucket.blob` is preferred here.
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Downloaded storage object {} from bucket {} to local file {}.".format(
            source_blob_name, bucket_name, destination_file_name
        )
    )
    
download_blob('openfoam-default-bucket', 'https://storage.cloud.google.com/openfoam-default-bucket/projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0/runs/cznonab01jnl0b55efd38c0b04b429a3a/case_files/log.simpleFoam', '/Users/scug/git/open_foam_cloud/openfoamcloud/log')


### Example to download specific folder from blob

In [3]:
from google.cloud import storage
import os
import logging

def findOccurrences(s, ch): # to find position of '/' in blob path ,used to create folders in local storage
    return [i for i, letter in enumerate(s) if letter == ch]

def download_from_bucket(bucket_name, blob_path, local_path):    
    # Create this folder locally
    if not os.path.exists(local_path):
        os.makedirs(local_path)        

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs=list(bucket.list_blobs(prefix=blob_path))

    startloc = 0
    for blob in blobs:
        startloc = 0
        folderloc = findOccurrences(blob.name.replace(blob_path, ''), '/') 
        if(not blob.name.endswith("/")):
            if(blob.name.replace(blob_path, '').find("/") == -1):
                downloadpath=local_path + '/' + blob.name.replace(blob_path, '')
                logging.info(downloadpath)
                blob.download_to_filename(downloadpath)
            else:
                for folder in folderloc:
                    
                    if not os.path.exists(local_path + '/' + blob.name.replace(blob_path, '')[startloc:folder]):
                        create_folder=local_path + '/' +blob.name.replace(blob_path, '')[0:startloc]+ '/' +blob.name.replace(blob_path, '')[startloc:folder]
                        startloc = folder + 1
                        os.makedirs(create_folder)
                    
                downloadpath=local_path + '/' + blob.name.replace(blob_path, '')

                blob.download_to_filename(downloadpath)
                logging.info(blob.name.replace(blob_path, '')[0:blob.name.replace(blob_path, '').find("/")])

    logging.info('Blob {} downloaded to {}.'.format(blob_path, local_path))


bucket_name = 'openfoam-default-bucket' # do not use gs://
blob_path = 'projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0/runs/cznonab01jnl0b55efd38c0b04b429a3a/case_files/system' # blob path in bucket where data is stored 
local_dir = '/Users/scug/git/open_foam_cloud/downloads' # folder in local
download_from_bucket(bucket_name, blob_path, local_dir)

### Misc functions to get info of project

In [29]:
# get project ids
# cloud_provider.get_project_ids()
project_id = 'test_project_scug_pvheadless_of10'
study_id = 'study_1'


# get project data 
project_data = cloud_provider.get_project_by_id(project_id)
print(project_data)

case_names = cloud_provider.get_case_names(project_id, study_id)
print(case_names)

case_path = cloud_provider.get_remote_case_path(project_id, study_id, case_names[2])
print(case_path)

study_path = cloud_provider.get_remote_parameter_study_path(project_id, study_id)   
print(study_path)

project_path = cloud_provider.get_remote_project_path(project_id)
print(project_path)

stud = cloud_provider.get_parameter_studies_ids(project_id)
print(stud)



id=UUID('52e078f0-bb57-42ef-9ee4-416756e720d7') path='projects/test_project_scug_pvheadless_of10' created_at=datetime.datetime(2024, 6, 11, 15, 21, 54, 484216) files_path='projects/test_project_scug_pvheadless_of10/base_files/pitzDaily' name='test_project_scug_pvheadless_of10'
['c3l9g6lu9ntr1', 'cufslmbfkvrp9', 'cznonab01jnl0']
projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0
projects/test_project_scug_pvheadless_of10/studies/study_1
projects/test_project_scug_pvheadless_of10
['study_1']


### Example to download specific folders from all study cases

In [50]:
from google.cloud import storage
import openfoamcloud.providers.google.google_provider as google_provider

cloud_provider = google_provider.GoogleCloud()

def download_many_blobs_with_transfer_manager(
    bucket_name, blob_names, destination_directory="", workers=8):
    """Download blobs in a list by name, concurrently in a process pool.

    The filename of each blob once downloaded is derived from the blob name and
    the `destination_directory `parameter. For complete control of the filename
    of each blob, use transfer_manager.download_many() instead.

    Directories will be created automatically as needed to accommodate blob
    names that include slashes.
    """

    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The list of blob names to download. The names of each blobs will also
    # be the name of each destination file (use transfer_manager.download_many()
    # instead to control each destination file name). If there is a "/" in the
    # blob name, then corresponding directories will be created on download.
    # blob_names = ["myblob", "myblob2"]

    # The directory on your computer to which to download all of the files. This
    # string is prepended (with os.path.join()) to the name of each blob to form
    # the full path. Relative paths and absolute paths are both accepted. An
    # empty string means "the current working directory". Note that this
    # parameter allows accepts directory traversal ("../" etc.) and is not
    # intended for unsanitized end user input.
    # destination_directory = ""

    # The maximum number of processes to use for the operation. The performance
    # impact of this value depends on the use case, but smaller files usually
    # benefit from a higher number of processes. Each additional process occupies
    # some CPU and memory resources until finished. Threads can be used instead
    # of processes by passing `worker_type=transfer_manager.THREAD`.
    # workers=8

    from google.cloud.storage import Client, transfer_manager

    storage_client = Client()
    bucket = storage_client.bucket(bucket_name)

    results = transfer_manager.download_many_to_path(
        bucket, blob_names, destination_directory=destination_directory, max_workers=workers
    )

    for name, result in zip(blob_names, results):
        # The results list is either `None` or an exception for each blob in
        # the input list, in order.

        if isinstance(result, Exception):
            print("Failed to download {} due to exception: {}".format(name, result))
        else:
            print("Downloaded {} to {}.".format(name, destination_directory + name))
            

def list_blobs_with_prefix(bucket_name, prefix, pattern, delimiter=None):
    """Lists all the blobs in the bucket that begin with the prefix.

    This can be used to list all blobs in a "folder", e.g. "public/".

    The delimiter argument can be used to restrict the results to only the
    "files" in the given "folder". Without the delimiter, the entire tree under
    the prefix is returned. For example, given these blobs:

        a/1.txt
        a/b/2.txt

    If you specify prefix ='a/', without a delimiter, you'll get back:

        a/1.txt
        a/b/2.txt

    However, if you specify prefix='a/' and delimiter='/', you'll get back
    only the file directly under 'a/':

        a/1.txt

    As part of the response, you'll also get back a blobs.prefixes entity
    that lists the "subfolders" under `a/`:

        a/b/
    """

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    # Note: The call returns a response only when the iterator is consumed.
    print("Blobs:")
    blob_list = []
    for blob in blobs:
        if pattern in blob.name:  # Filter blobs who match the pattern
            print(blob.name)
            blob_list.append(blob.name)

    if delimiter:
        print("Prefixes:")
        for prefix in blobs.prefixes:
            print(prefix)

    return blob_list


bucket_name = 'openfoam-default-bucket' # do not use gs://
project_id = 'test_project_scug_pvheadless_of10'  # Project name given while creating the project
study_id = 'study_1' # Study name given while creating the project

case_names = cloud_provider.get_case_names(project_id, study_id)

for case in case_names:
    case_path = cloud_provider.get_remote_case_path(project_id, study_id, case)
    
    blob_list = list_blobs_with_prefix(bucket_name, case_path+'/', 'postProcessing', delimiter=None)
    print(blob_list)

    download_many_blobs_with_transfer_manager(
        bucket_name, blob_list, destination_directory="/Users/scug/git/open_foam_cloud/downloads/", workers=8)

Blobs:
[]
Blobs:
[]
Blobs:
projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0/runs/cznonab01jnl0b55efd38c0b04b429a3a/case_files/postProcessing/
projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0/runs/cznonab01jnl0b55efd38c0b04b429a3a/case_files/postProcessing/streamlines/
projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0/runs/cznonab01jnl0b55efd38c0b04b429a3a/case_files/postProcessing/streamlines/0/
projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0/runs/cznonab01jnl0b55efd38c0b04b429a3a/case_files/postProcessing/streamlines/0/tracks.vtk
projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0/runs/cznonab01jnl0b55efd38c0b04b429a3a/case_files/postProcessing/streamlines/100/
projects/test_project_scug_pvheadless_of10/studies/study_1/cases/cznonab01jnl0/runs/cznonab01jnl0b55efd38c0b04b429a3a/case_files/postProcessing/streamlines/100/tracks.vtk
projects/te